In [23]:
import pandas as pd
import numpy as np


In [24]:
pd.set_option("display.max_columns", None)

data = pd.read_csv('../../data/processed/la_liga_data.csv')
data['Time'] = pd.to_datetime(data['Time'])
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5520 entries, 0 to 5519
Data columns (total 165 columns):
 #    Column                Dtype         
---   ------                -----         
 0    Time                  datetime64[ns]
 1    Comp                  object        
 2    Round                 int64         
 3    Day                   int64         
 4    Venue                 int64         
 5    Result                object        
 6    GF                    float64       
 7    GA                    float64       
 8    Opponent              object        
 9    xG                    float64       
 10   xGA                   float64       
 11   Captain               object        
 12   Formation             object        
 13   Opp Formation         object        
 14   Referee               object        
 15   Standard__Gls         int64         
 16   Standard__Sh          float64       
 17   Standard__SoT         float64       
 18   Standard__SoT%        floa

In [25]:
# data.head()

In [26]:
# data.drop(columns=['Comp', 'Day', 'Captain', 'Formation', 'Opp Formation', 'Referee'], inplace=True)

In [27]:
# for col in data.select_dtypes(include=[np.number]).columns:
#     data[col] = data[col].fillna(data.groupby('Team')[col].transform('mean'))
#     # df2[cols] = df2[cols].fillna(df2.groupby('Team')[cols].mean())

#### Get current form features

In [28]:
current_form_features = pd.read_csv('eda_data/selected/all_feature_names_rest.csv')
window_size = 5
current_form_features = set(current_form_features['Feature_Name'].values)
# current_form_features.remove('Performance__GA')
# current_form_features.remove('Standard__Gls')
current_form_features = list(current_form_features) + ['Sum_points'] 
current_form_features

['xG',
 'Touches__Att Pen',
 'Performance__Crs',
 'xGA',
 'PPA',
 'Carries__CPA',
 'Performance__PKcon',
 'Pass Types__TB',
 'KP',
 'Pass Types__Crs',
 'Performance__PKwon',
 'Ast',
 'Sum_points']

In [29]:
# # Sort the data by Team, Season, Round, and Time
# data = data.sort_values(['Team', 'Season', 'Round', 'Time'])
# train_data = data[['Time', 'Season', 'Venue', 'Round', 'Team', 'Opponent', 'GF', 'GA', 'Result', 'Points']].copy()

In [30]:
def exponentially_weighted_moving_average(data):

    alpha = 0.9
    ewma = [data[0]]  # Initialize with the first data point
    for i in range(1, len(data)):
        An = alpha * data[i] + (1 - alpha) * ewma[-1]
        ewma.append(An)

    return ewma[-1]

In [31]:
# df = train_data.copy()

In [32]:
# # Step 1: Calculate total points per team for each season
# season_points = df.groupby(['Season', 'Team'])['Points'].sum().reset_index()
# season_points = season_points.rename(columns={'Points': 'TotalPoints'})

# # Step 2: Prepare a DataFrame with the lowest points for each season
# lowest_points_per_season = season_points.groupby('Season')['TotalPoints'].min().reset_index()
# lowest_points_per_season = lowest_points_per_season.rename(columns={'TotalPoints': 'LowestPoints'})

# # Step 3: Merge last season's data into the current season
# season_points['LastSeason'] = season_points['Season'] - 1

# # Left join to bring last season's points
# result = season_points.merge(
#     season_points[['Season', 'Team', 'TotalPoints']],
#     left_on=['LastSeason', 'Team'],
#     right_on=['Season', 'Team'],
#     how='left',
#     suffixes=('', '_LastSeason')
# )

# # Step 4: Merge lowest points for last season
# result = result.merge(
#     lowest_points_per_season,
#     left_on='LastSeason',
#     right_on='Season',
#     how='left'
# )

# # Step 5: Assign LastSeasonPoints based on the condition
# result['LastSeasonPoints'] = result['TotalPoints_LastSeason'].fillna(result['LowestPoints'])

# # Instead of dropping columns and using apply, directly modify 'LastSeasonPoints' based on conditions
# result['LastSeasonPoints'] = result.apply(lambda row: 0 if row['Season_x'] == 2017 else row['LastSeasonPoints'], axis=1)

# # Cleanup columns - Only drop unnecessary columns
# result = result.drop(columns=['Season_y', 'LastSeason'])
# result = result.rename(columns={'Season_x': 'Season'}) # Rename 'Season_x' to 'Season'

In [33]:
# train_data = train_data.merge(result[['Team', 'Season', 'LastSeasonPoints']], on=['Team', 'Season'], how='left')

In [34]:
# train_data.tail()

In [35]:
# # Calculate the current points
# train_data['CurrentPoints'] = data.groupby(['Team', 'Season'])['Points'].transform(
#     lambda x: x.shift().cumsum()
# ).fillna(0)

# train_data['Sum_points'] = train_data['CurrentPoints'] + train_data['LastSeasonPoints']

In [36]:
# train_data.tail()

In [37]:
# data = data.merge(train_data[['Team', 'Season', 'Sum_points']], on=['Team', 'Season'], how='left')

In [38]:

# # # Calculate rolling averages for the last 5 matches
# # for feature in current_form_features:

# #     train_data[f"{feature}_avg_last5"] = data.groupby('Team')[feature].transform(
# #         lambda x: x.shift().rolling(window=window_size, min_periods=1).apply(
# #             exponentially_weighted_moving_average, raw=True
# #         )
# #     ).fillna(0)


# # Step 1: Calculate rolling averages for the last 5 matches for each feature
# for feature in current_form_features:
#     train_data[f"{feature}_avg_last5"] = data.groupby('Team')[feature].transform(
#         lambda x: x.shift().rolling(window=window_size, min_periods=1).apply(
#             exponentially_weighted_moving_average, raw=True
#         )
#     ).fillna(0)

# # Step 2: Prepare the opponent's rolling average
# # Rename the columns for merging (e.g., "Team" -> "Opponent" and avg_last5 -> Opponent's avg_last5)
# for feature in current_form_features:
#     opponent_avg_col = f"{feature}_avg_last5"

#     # Copy the required columns and rename for the merge
#     opponent_rolling_avg = train_data[['Time', 'Team', opponent_avg_col]].copy()
#     opponent_rolling_avg = opponent_rolling_avg.rename(columns={
#         'Team': 'Opponent',
#         opponent_avg_col: f"{feature}_opponent_avg_last5"
#     })

#     # Step 3: Merge the opponent's rolling average back into the original DataFrame
#     train_data = train_data.merge(
#         opponent_rolling_avg,
#         on=['Time', 'Opponent'],
#         how='left'
#     )

# # Fill NaN values with 0 for any missing opponents' averages
# train_data = train_data.fillna(0)

# # Display the updated DataFrame
# train_data.tail()


#     # if feature != 'Performance__CS':
#     #     train_data[f"{feature}_avg_last5"] = data.groupby('Team')[feature].transform(
#     #         lambda x: x.shift().rolling(window=window_size, min_periods=1).mean()
#     #     )
#     # else:
#     #     train_data[f"{feature}_avg_last5"] = data.groupby('Team')[feature].transform(
#     #         lambda x: x.shift().rolling(window=window_size, min_periods=1).sum()
#     #     )

In [39]:
# train_data.tail()

In [40]:
# train_data[(train_data['Team'] == 'Real Madrid') & (train_data['Season'] == 2023) &(train_data['Opponent'] == 'Barcelona')]

In [41]:
# train_data[(train_data['Team'] == 'Barcelona') & (train_data['Season'] == 2023) &(train_data['Opponent'] == 'Real Madrid')]

In [42]:
def get_the_train_data(data, current_form_features):

    data.drop(columns=['Comp', 'Day', 'Captain', 'Formation', 'Opp Formation', 'Referee'], inplace=True)
    for col in data.select_dtypes(include=[np.number]).columns:
        data[col] = data[col].fillna(data.groupby('Team')[col].transform('mean'))
    window_size = 5
    data = data.sort_values(['Team', 'Season', 'Round', 'Time'])
    train_data = data[['Time', 'Season', 'Venue', 'Round', 'Team', 'Opponent', 'GF', 'GA', 'Result', 'Points']].copy()


    df = train_data.copy()
    # Step 1: Calculate total points per team for each season
    season_points = df.groupby(['Season', 'Team'])['Points'].sum().reset_index()
    season_points = season_points.rename(columns={'Points': 'TotalPoints'})

    # Step 2: Prepare a DataFrame with the lowest points for each season
    lowest_points_per_season = season_points.groupby('Season')['TotalPoints'].min().reset_index()
    lowest_points_per_season = lowest_points_per_season.rename(columns={'TotalPoints': 'LowestPoints'})

    # Step 3: Merge last season's data into the current season
    season_points['LastSeason'] = season_points['Season'] - 1

    # Left join to bring last season's points
    result = season_points.merge(
        season_points[['Season', 'Team', 'TotalPoints']],
        left_on=['LastSeason', 'Team'],
        right_on=['Season', 'Team'],
        how='left',
        suffixes=('', '_LastSeason')
    )

    # Step 4: Merge lowest points for last season
    result = result.merge(
        lowest_points_per_season,
        left_on='LastSeason',
        right_on='Season',
        how='left'
    )

    # Step 5: Assign LastSeasonPoints based on the condition
    result['LastSeasonPoints'] = result['TotalPoints_LastSeason'].fillna(result['LowestPoints'])

    # Instead of dropping columns and using apply, directly modify 'LastSeasonPoints' based on conditions
    result['LastSeasonPoints'] = result.apply(lambda row: 0 if row['Season_x'] == 2017 else row['LastSeasonPoints'], axis=1)

    # Cleanup columns - Only drop unnecessary columns
    result = result.drop(columns=['Season_y', 'LastSeason'])
    result = result.rename(columns={'Season_x': 'Season'}) # Rename 'Season_x' to 'Season'

    train_data = train_data.merge(result[['Team', 'Season', 'LastSeasonPoints']], on=['Team', 'Season'], how='left')

    # Calculate the current points
    train_data['CurrentPoints'] = data.groupby(['Team', 'Season'])['Points'].transform(
        lambda x: x.shift().cumsum()
    ).fillna(0)

    train_data['Sum_points'] = train_data['CurrentPoints'] + train_data['LastSeasonPoints']
    data = data.merge(train_data[['Team', 'Season', 'Sum_points']], on=['Team', 'Season'], how='left')
    # Step 1: Calculate rolling averages for the last 5 matches for each feature
    for feature in current_form_features:
        train_data[f"{feature}_avg_last5"] = data.groupby('Team')[feature].transform(
            lambda x: x.shift().rolling(window=window_size, min_periods=1).apply(
                exponentially_weighted_moving_average, raw=True
            )
        ).fillna(0)

    # Step 2: Prepare the opponent's rolling average
    # Rename the columns for merging (e.g., "Team" -> "Opponent" and avg_last5 -> Opponent's avg_last5)
    for feature in current_form_features:
        opponent_avg_col = f"{feature}_avg_last5"

        # Copy the required columns and rename for the merge
        opponent_rolling_avg = train_data[['Time', 'Team', opponent_avg_col]].copy()
        opponent_rolling_avg = opponent_rolling_avg.rename(columns={
            'Team': 'Opponent',
            opponent_avg_col: f"{feature}_opponent_avg_last5"
        })

        # Step 3: Merge the opponent's rolling average back into the original DataFrame
        train_data = train_data.merge(
            opponent_rolling_avg,
            on=['Time', 'Opponent'],
            how='left'
        )

    # Fill NaN values with 0 for any missing opponents' averages
    train_data = train_data.fillna(0)
    return train_data

In [43]:
train_data = get_the_train_data(data, current_form_features)

In [44]:
train_data.tail()

,Time,Season,Venue,Round,Team,Opponent,GF,GA,Result,Points,LastSeasonPoints,CurrentPoints,Sum_points,xG_avg_last5,Touches__Att Pen_avg_last5,Performance__Crs_avg_last5,xGA_avg_last5,PPA_avg_last5,Carries__CPA_avg_last5,Performance__PKcon_avg_last5,Pass Types__TB_avg_last5,KP_avg_last5,Pass Types__Crs_avg_last5,Performance__PKwon_avg_last5,Ast_avg_last5,Sum_points_avg_last5,xG_opponent_avg_last5,Touches__Att Pen_opponent_avg_last5,Performance__Crs_opponent_avg_last5,xGA_opponent_avg_last5,PPA_opponent_avg_last5,Carries__CPA_opponent_avg_last5,Performance__PKcon_opponent_avg_last5,Pass Types__TB_opponent_avg_last5,KP_opponent_avg_last5,Pass Types__Crs_opponent_avg_last5,Performance__PKwon_opponent_avg_last5,Ast_opponent_avg_last5,Sum_points_opponent_avg_last5
5515,2024-09-22 18:30:00,2024,1,6,Villarreal,Barcelona,1.0,5.0,L,0,53.0,8.0,61.0,1.0,10.0,19.0,1.0,3.0,0.0,0.0,0.0,5.0,19.0,0.0,1.0,47.9952,0.7,17.0,19.0,0.7,8.0,2.0,0.0,2.0,8.0,19.0,0.0,0.0,2.9799
5516,2024-09-26 19:00:00,2024,0,7,Villarreal,Espanyol,2.0,1.0,W,3,53.0,12.0,65.0,1.0,10.0,19.0,1.0,3.0,0.0,0.0,0.0,5.0,19.0,0.0,1.0,47.9995,0.8,11.0,12.0,1.1,3.0,4.0,0.0,0.0,5.0,12.0,0.0,1.0,47.0397
5517,2024-09-30 21:00:00,2024,1,8,Villarreal,Las Palmas,3.0,1.0,W,3,53.0,24.0,77.0,1.0,10.0,19.0,1.0,3.0,0.0,0.0,0.0,5.0,19.0,0.0,1.0,46.2000,0.8,23.0,25.0,1.9,12.0,0.0,0.0,0.0,9.0,25.0,0.0,1.0,48.2910
5518,2024-10-05 21:00:00,2024,0,9,Villarreal,Real Madrid,0.0,2.0,L,0,53.0,6.0,59.0,1.0,10.0,19.0,1.0,3.0,0.0,0.0,0.0,5.0,19.0,0.0,1.0,44.2200,0.2,9.0,15.0,0.7,1.0,3.0,0.0,1.0,2.0,15.0,0.0,0.0,54.1895
5519,2024-10-20 18:30:00,2024,1,10,Villarreal,Getafe,1.0,1.0,D,1,53.0,3.0,56.0,1.0,10.0,19.0,1.0,3.0,0.0,0.0,0.0,5.0,19.0,0.0,1.0,43.1220,1.2,17.0,14.0,3.1,4.0,2.0,0.0,0.0,5.0,14.0,0.0,1.0,51.7875


In [49]:
train_data = train_data.drop(['LastSeasonPoints', 'CurrentPoints', 'Sum_points'], axis=1)

In [50]:
train_data.to_csv('train_data.csv')